### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [ ]:
library(tibble)

In [ ]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

In [ ]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [ ]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [ ]:
head(df,2)

In [ ]:
df$flag <- FALSE
df$true_false <- FALSE


In [ ]:
head(df, 2)

## <font color = yellow>1. Create data & dfs

In [ ]:
create_data <- function () {   
   
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$date <- as.Date(df$date, origin = "1899-12-30")
 
    df$flag <- FALSE
    df$true_false <- FALSE
    
    sale_records <- data.frame() 

    # create dfs from filtered df
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    b <- buys[1,]
    s <- sells[1,]
    
    
    # Assign variables to the global environment
    assign("df", df, envir = .GlobalEnv)
    assign("buys", buys, envir = .GlobalEnv)
    assign("sells", sells, envir = .GlobalEnv)
    assign("b", b, envir = .GlobalEnv)
    assign("s", s, envir = .GlobalEnv)
    
    }

create_data()


In [ ]:
dim(df)


## <font color = lime> Examining data and dfs

In [ ]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 2)
head(sells,2)

In [ ]:
dim(b)
b

## <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [ ]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

In [ ]:
last_row <- tail(buys, n = 1)
last_row

#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [ ]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function() {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) { 
            
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$sold_cf_pps <- b$pps
            b$sold_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE         
    
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {            
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
          
 #           s$type <- "SOLD"            
            }
            
        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

In [ ]:
# print(t(updated_b))

### <font color = yellow>3. Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )  MOVED

In [ ]:
dim(buys)
dim(sells)

In [ ]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    dim(s1)
    # save_buy_one <- buys[-1,]

    buys <- buys[-1, ]
    sells <- sells[-1, ]
    
    return(list(b1, s1, buys, sells))
        }

# copy_snip_list <- copy_snip(buys, sells) 

# b1 <- copy_snip_list[[1]]
# s1 <- copy_snip_list[[2]]
# buys <- copy_snip_list[[3]]
# sells <- copy_snip_list[[4]]

# after
# dim(buys)
# dim(sells)
# b1
# s1

### <font color = yellow>4. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [ ]:
df_name <- "iris"  # Replace with your data frame name
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)
         

In [ ]:
df_name <- "sale_records"
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

In [ ]:
# sale_records <- data.frame()
# sale_records <- rbind(sale_records, updated_b)
# sale_records
                      

In [ ]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    cat("Newly created sale_records with updated_b data. ", "\n")
    sale_records <- data.frame()
    sale_records <- rbind(sale_records, updated_b)    
}
sale_records

In [ ]:
updated_b

In [ ]:
dim(buys)
dim(sells)

### <font color = yellow> 5. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction


#### <font color = lime> How it's done

In [ ]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

In [ ]:
first_row <- head(sells, n = 1)
first_row

In [ ]:
last_row <- tail(sale_records, n = 1)
last_row

In [ ]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_withcf <- function(buys, sells) {

    last_row <- tail(sale_records, n = 1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    
    #  b > s  ... buy c/f
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
        b1 <- last_row
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        test <- test %>% add_row(b1, .before = 1)
        buys <- test

       
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
       
        cat("\n", "There is Sold carryforward. Update sells ", last_row$sold_cf_quan, "\n")
            cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/f"
        b1 <- last_row
       
        b1$type <- "Sell CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- sells
        test <- test %>% add_row(b1, .before = 1)
        sells <- test
                      
        }

    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ] w/ sold c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... Sells c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_withcf. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
return_list <- update_withcf(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)


### <font color = yellow>3. Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [ ]:
buys[1,]

In [ ]:
dim(buys)
dim(sells)

In [ ]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    dim(s1)
    # save_buy_one <- buys[-1,]

    buys <- buys[-1, ]
    sells <- sells[-1, ]
    
    return(list(b1, s1, buys, sells))
        }

copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
b1
s1

#### <font color = lime> QC: review buys-sells for updated carryforward
    
#### When c/f exists, that c/f (type) must be inserted into the next row of either buys or sells

In [ ]:
head(buys,2)
head(sells,2)

#### <font color = lime> QC: review sale_record for accuracy

In [ ]:
sale_records

### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [ ]:
stopps #  Old code

In [ ]:
dim(sale_records)

## <font color = yellow> Proof of Concept above. Make it loop 2 x.

#### <font color = yellow>1. Create data & dfs

In [ ]:
# create_data <- function () {   
   
#     df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
#     df$date <- as.Date(df$date, origin = "1899-12-30")
 
#     df$flag <- FALSE
#     df$true_false <- FALSE
    
#     sale_records <- data.frame() 

#     # create dfs from filtered df
#     buys <- df[df$type == 'Buy', ]  # Filter buys
#     sells <- df[df$type == 'Sell', ]  # Filter sells
#     b <- buys[1,]
#     s <- sells[1,]
    
    
#     # Assign variables to the global environment
#     assign("df", df, envir = .GlobalEnv)
#     assign("buys", buys, envir = .GlobalEnv)
#     assign("sells", sells, envir = .GlobalEnv)
#     assign("b", b, envir = .GlobalEnv)
#     assign("s", s, envir = .GlobalEnv)
    
#     }

#  create_data()

In [ ]:
# fct call   ... in loop 2, only need new b and s
b <- buys[1,]
s <- sells[1,]

### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [ ]:
b

In [ ]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function() {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) { 
            
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$sold_cf_pps <- b$pps
            b$sold_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE         
    
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {            
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
          
 #           s$type <- "SOLD"            
            }
            
        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    
    

In [ ]:
# print(t(updated_b))

### <font color = yellow>3. Create a new variable from buys/sells row 1, then delete row 1 from buys and sells
    
#### issues ... cannot snip the previous addition of c/f data

In [ ]:
buys[1, ]
sells[1, ]

In [ ]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    dim(s1)
    # save_buy_one <- buys[-1,]
    
    if(buys$flag == FALSE ) {
        cat("There is NO c/f ... t/f snip buys ", "\n")
         buys <- buys[-1, ]
      
    } else{
        cat("There is c/f ... t/f do NOT snip buys ", "\n")
         
    }
     if(sells$true_false == TRUE ) {   
        cat("There is c/f ... t/f, do NOT snip sells ", "\n")
    } else {
          sells <- sells[-1, ]
         }
    
    return(list(b1, s1, buys, sells))
        }

copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
b1
s1

In [ ]:
df_name <- "sale_records"
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

### <font color = yellow>4. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [ ]:
# sale_records <- data.frame()
# sale_records <- rbind(sale_records, updated_b)
sale_records
                      

In [ ]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n")
    sale_records <- rbind(sale_records, updated_b)    
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
   
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    
}
sale_records

### <font color = yellow> 5. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction

In [ ]:
dim(buys)
dim(sells)

In [ ]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_withcf <- function(buys, sells) {

    last_row <- tail(sale_records, n = 1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    
    #  b > s  ... buy c/f
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
        b1 <- last_row
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        test <- test %>% add_row(b1, .before = 1)
        buys <- test

       
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
       
        cat("\n", "There is Sold carryforward. Update sells ", last_row$sold_cf_quan, "\n")
            cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/f"
        b1 <- last_row
       
        b1$type <- "Sell CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- sells
        test <- test %>% add_row(b1, .before = 1)
        sells <- test
                      
        }

    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ] w/ sold c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... Sells c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_withcf. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
return_list <- update_withcf(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)


In [ ]:
buys[1:2,]
sells[1:2,]

#### <font color = lime> QC: review buys-sells for updated carryforward
    
#### When c/f exists, that c/f (type) must be inserted into the next row of either buys or sells

In [ ]:
head(sells, 2)

In [ ]:
head(buys, 2)

In [ ]:
sale_records

In [ ]:
stopps

### <font color = lime > 1.  b and s

In [ ]:
b <- buys[1,]
s <- sells[1,]
b
s

### <font color = lime> 2.  updated_b  <-  b

In [ ]:
# update b based on Sale Type: 

update_b <- function(b) {
    
     cat("Inside update_b, test for: $quan < s$quan && b$type == 'Buy CF': ", "\n")
    
        if(b$quan < s$quan && b$type == 'Buy CF') {
            cat("True", "\n")
            }

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        sold_cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
         #  b > s ... restricted to Buy only
        if ( b$quan > s$quan && b$type == "Buy") {  
            
            # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "Buy-CF"           
        
            
        }else if( b$quan < s$quan && b$type == 'Buy CF') {    

                # protect b from errors
            temp <- b  

            temp$type <- "Buy-2xCF"           
            temp$sell_quan <- temp$quan
            temp$sell_pps <- sale_records$sell_pps[1]
            temp$sell_basis <- temp$sell_quan * sale_records$sell_pps
            temp$profit_loss <- temp$sell_basis - temp$basis

            temp$Sell_cf  <-  sale_records$sell_quan - sale_records$sold_cf_quan 
            temp$Sell_cf_pps   <- sale_records$sell_pps
            # update b with temp
            b <- temp
            s$type <- "Buy-2xCF"       


               # ( s > b ), has sell carryforward
        }else if(s$quan > b$quan) {            
            
            b$sale_quan <- b$quan
           # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps
        
            # Sell c/f
            S_cf_quan <- s$quan - b$quan           
            b$Sell_cf <-S_cf_quan
            b$Sell_cf_pps <- s$pps
            b$Sell_cf_basis <- S_cf_quan * s$pps
            
     
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis          
            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {            
           
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
           # b$sale <- "sale1"
            s$type <- "SOLD"
            
            }
    
        return(b)   
}  # fct

updated_b <- update_b(b)
updated_b  
    

### <font color = lime>3.  buys <- buy[-1, ], sells <- sells[-1, ]
    
##### Nec to start loop at the first (unprocessed rows in buys / sells at the top of the loop

In [ ]:
save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)

save_sell_one <- sells [1, ]
save_buy_one <- buys[-1,]

buys <- buys[-1, ]
sells <- sells[-1, ]

# nrow(buys)

### <font color = lime> 4. sale_records <- rbind(sale_records, updated_b)

In [ ]:
sale_records <- rbind(sale_records, updated_b)
sale_records